In [2]:
import numpy as np
import tools
import torch
import torch.nn as nn
import torch.nn.functional as F

# Some magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
Languages = {
    'en':'English', 
    'es':'Spanish', 
    'pt':'Portuguese', 
    'gl':'Galician', 
    'eu':'Basque', 
    'ca':'Catalan', 
    'fr':'French', 
    'it':'Italian', 
    'de':'German'
}

## Load the Data, Calculate Vocabulary and Perplexity

In [14]:
# Load Data
data = tools.Data()
print('Size of the vocabulary: %d characters' % len(data.vocab))

Size of the vocabulary: 509 characters


In [16]:
preplexity = data.get_perplexity()
print('Preplexity measurement is %.2f' % preplexity)

Preplexity measurement is 34.11


In [24]:
print('Percent of Invalid Characters - Train: %.5f%%' 
    % ((data.train_freq[data.vocab.index('<N>')] / data.train_freq.sum()) * 100.0))
print('Percent of Invalid Characters - Val: %.5f%%' 
    % ((data.val_freq[data.vocab.index('<N>')] / data.val_freq.sum()) * 100.0))

Percent of Invalid Characters - Train: 0.04622%
Percent of Invalid Characters - Val: 0.05987%


## Model (Doruk)

## Model (John)